# Learning Prefect: Hello World Flow

This notebook will teach you the basics of Prefect by creating a simple "Hello World" flow.

**What you'll learn:**
1. How to create a Prefect `@task` - a unit of work
2. How to create a Prefect `@flow` - orchestrates tasks
3. How to run flows and see them in the Prefect UI

## Step 1: Import Prefect and configure the API

In [3]:
import os
from prefect import flow, task

# Configure to use your local Prefect server
os.environ["PREFECT_API_URL"] = "http://localhost:4200/api"

print("✅ Prefect imported and configured to use:", os.environ["PREFECT_API_URL"])

✅ Prefect imported and configured to use: http://localhost:4200/api


## Step 2: Create a simple task

A **task** is a Python function decorated with `@task`. Tasks are the building blocks of flows.

In [5]:
@task
def print_hello():
    """A simple task that prints Hello World"""
    message = "Hello World from Prefect!"
    print(message)
    return message

# Note: Tasks must be called from within a flow!
# If you want to test the function directly, use: print_hello.fn()
print("✅ Task 'print_hello' created successfully!")
print("Note: Tasks can only be called from within flows")

✅ Task 'print_hello' created successfully!
Note: Tasks can only be called from within flows


## Step 3: Create a flow that uses the task

A **flow** is the main orchestrator. It calls tasks and manages the workflow.

### First, let's verify connection to the Prefect server

In [7]:
import requests

# Test connection to Prefect server
try:
    response = requests.get("http://localhost:4200/api/health")
    if response.status_code == 200:
        print("✅ Successfully connected to Prefect server!")
        print(f"Server response: {response.json()}")
    else:
        print(f"❌ Server returned status code: {response.status_code}")
except Exception as e:
    print(f"❌ Could not connect to server: {e}")

✅ Successfully connected to Prefect server!
Server response: True


### Now let's run a simple flow!

For this to work with the external Prefect server, we'll write the flow to a Python file and run it.

In [8]:
# Let's use the existing prefect-main.py instead
# First, let's run it!
import subprocess

result = subprocess.run(
    ["uv", "run", "python", "prefect-main.py"],
    cwd="/Users/jichong/projects/playground/BOS/prefect-poc",
    capture_output=True,
    text=True,
    env={**os.environ, "PREFECT_API_URL": "http://localhost:4200/api"}
)

print("Flow Output:")
print(result.stdout)
if result.stderr:
    print("\nWarnings/Errors:")
    print(result.stderr)

Flow Output:
What is your favorite number?
Favorite number: 42
Customer IDs: ['customer30', 'customer71', 'customer27', 'customer56', 'customer72', 'customer95', 'customer47', 'customer75', 'customer71', 'customer28']


Warnings/Errors:
19:18:10.244 | INFO    | prefect.engine - Created flow run 'organic-spoonbill' for flow 'my-favorite-function'
19:18:10.245 | INFO    | Flow run 'organic-spoonbill' - View at http://localhost:4200/flow-runs/flow-run/eddf86db-c258-473a-b62d-c85f973ed21b
19:18:10.327 | INFO    | Flow run 'organic-spoonbill' - Created task run 'get_customer_ids-0' for task 'get_customer_ids'
19:18:10.328 | INFO    | Flow run 'organic-spoonbill' - Executing 'get_customer_ids-0' immediately...
19:18:10.400 | INFO    | Task run 'get_customer_ids-0' - Finished in state Completed()
19:18:10.418 | INFO    | Flow run 'organic-spoonbill' - Finished in state Completed()



### Check the Prefect UI!

🎉 **Success!** Your flow ran and was tracked by the Prefect server.

**What just happened:**
1. The flow `my-favorite-function` executed successfully
2. It created and executed the task `get_customer_ids`
3. All metadata was sent to your Prefect server at http://localhost:4200
4. You can see the run at the URL shown above (http://localhost:4200/flow-runs/flow-run/...)

### Query flow runs from the Prefect API

Now let's see how to programmatically query the flows and flow runs:

In [9]:
import asyncio
from prefect.client.orchestration import PrefectClient

async def get_recent_flow_runs():
    """Query recent flow runs from the Prefect server"""
    async with PrefectClient(api="http://localhost:4200/api") as client:
        # Get recent flow runs
        runs = await client.read_flow_runs(limit=5)
        
        print(f"Found {len(runs)} recent flow runs:\n")
        for run in runs:
            print(f"  • Flow: {run.name}")
            print(f"    State: {run.state.type if run.state else 'Unknown'}")
            print(f"    Start time: {run.start_time}")
            print(f"    ID: {run.id}")
            print()
        
        return runs

# Run the async function
runs = await get_recent_flow_runs()

Found 1 recent flow runs:

  • Flow: organic-spoonbill
    State: COMPLETED
    Start time: 2025-10-14T11:18:10.248871+00:00
    ID: eddf86db-c258-473a-b62d-c85f973ed21b



# Part 2: Deployments and Work Pools

Now let's learn about **Deployments** and **Work Pools** - how to let the Prefect server schedule and orchestrate your flows!

## Concepts:

- **Deployment**: Packages your flow for remote execution and scheduling
- **Work Pool**: A queue where the server sends flow runs to be executed
- **Worker**: An agent that pulls work from the pool and executes it

Think of it like a restaurant:
- **Deployment** = Menu item (your flow, ready to be ordered)
- **Work Pool** = Kitchen ticket queue
- **Worker** = Chef who takes tickets and cooks

## Step 1: Create a Work Pool

First, we need to create a work pool. A work pool is like a job queue where the server puts flow runs that need to be executed.

In [10]:
# Create a work pool using the Prefect CLI
import subprocess

# Create a process work pool (runs flows as local processes)
result = subprocess.run(
    ["uv", "run", "prefect", "work-pool", "create", "my-local-pool", "--type", "process"],
    capture_output=True,
    text=True,
    env={**os.environ, "PREFECT_API_URL": "http://localhost:4200/api"}
)

print("Work Pool Creation:")
print(result.stdout)
if result.stderr:
    print(result.stderr)

Work Pool Creation:
Created work pool 'my-local-pool'.




## Step 2: Create a Deployment

Now let's deploy our flow. This tells the Prefect server about the flow and how to run it.

In [16]:
# For demonstration, let's show how to create a deployment via the command line
# This is the recommended approach for Prefect 2.13

deployment_guide = """
To create the deployment, run these commands in your terminal:

cd /Users/jichong/projects/playground/BOS/prefect-poc
export PREFECT_API_URL=http://localhost:4200/api

# Method 1: Quick deploy (interactive)
uv run prefect deploy deployed_flow.py:data_pipeline \\
    --name my-data-pipeline \\
    --pool my-local-pool

# Method 2: Or use Python code
"""

print("📋 Deployment Creation Guide")
print("=" * 70)
print(deployment_guide)

# For this demo, let's verify the work pool exists and is ready
async def check_work_pool():
    async with PrefectClient(api="http://localhost:4200/api") as client:
        try:
            pools = await client.read_work_pools()
            print(f"\n✅ Work pools available on server:")
            for pool in pools:
                print(f"   • {pool.name} (type: {pool.type})")
            return pools
        except Exception as e:
            print(f"❌ Error checking work pools: {e}")
            return []

pools = await check_work_pool()

print("\n💡 Next step: Start a worker (see next cell)")

📋 Deployment Creation Guide

To create the deployment, run these commands in your terminal:

cd /Users/jichong/projects/playground/BOS/prefect-poc
export PREFECT_API_URL=http://localhost:4200/api

# Method 1: Quick deploy (interactive)
uv run prefect deploy deployed_flow.py:data_pipeline \
    --name my-data-pipeline \
    --pool my-local-pool

# Method 2: Or use Python code


✅ Work pools available on server:
   • default-agent-pool (type: prefect-agent)
   • my-local-pool (type: process)

💡 Next step: Start a worker (see next cell)


## Step 3: Start a Worker

A worker is the "chef" that pulls jobs from the work pool and executes them. Without a worker, deployments won't run!

**Note:** We'll start the worker in the background so it can pick up flow runs.

In [17]:
# We'll show you the command to start a worker
# In practice, you'd run this in a separate terminal

worker_command = """
# Run this in a separate terminal:
cd /Users/jichong/projects/playground/BOS/prefect-poc
export PREFECT_API_URL=http://localhost:4200/api
uv run prefect worker start --pool my-local-pool
"""

print("🔧 To start a worker, run this command in a SEPARATE TERMINAL:")
print(worker_command)
print("\nThe worker will:")
print("  1. Connect to the Prefect server")
print("  2. Poll the 'my-local-pool' work pool for flow runs")
print("  3. Execute any flows that are scheduled")
print("  4. Report results back to the server")
print("\n⚠️  Leave the worker running to process flows!")

🔧 To start a worker, run this command in a SEPARATE TERMINAL:

# Run this in a separate terminal:
cd /Users/jichong/projects/playground/BOS/prefect-poc
export PREFECT_API_URL=http://localhost:4200/api
uv run prefect worker start --pool my-local-pool


The worker will:
  1. Connect to the Prefect server
  2. Poll the 'my-local-pool' work pool for flow runs
  3. Execute any flows that are scheduled
  4. Report results back to the server

⚠️  Leave the worker running to process flows!


## Step 4: Trigger a Flow Run from the Server

Now that we have a deployment, we can trigger it through the server API (not by running the Python file directly!).

In [ ]:
# Trigger a flow run via the API
# This creates a flow run that gets queued in the work pool

result = subprocess.run([
    "uv", "run", "prefect", "deployment", "run",
    "data-pipeline/my-data-pipeline",  # flow-name/deployment-name
    "--param", "source=production-api",
    "--param", "destination=warehouse"
],
    cwd="/Users/jichong/projects/playground/BOS/prefect-poc",
    capture_output=True,
    text=True,
    env={**os.environ, "PREFECT_API_URL": "http://localhost:4200/api"}
)

print("Flow Run Triggered!")
print(result.stdout)
if result.stderr:
    print("\nMessages:")
    print(result.stderr)
    
print("\n" + "="*60)
print("What happens now:")
print("="*60)
print("1. ✅ Server creates a flow run and puts it in the work pool queue")
print("2. ⏳ Worker (if running) picks up the flow run")
print("3. 🚀 Worker executes the flow on its machine")
print("4. 📊 Worker reports progress back to the server")
print("5. 🎉 You can watch it in the UI at http://localhost:4200")

## Summary: How Deployments Work

### The Full Architecture:

```
┌─────────────────────────────────────────────────────┐
│  YOU (Developer)                                    │
│                                                     │
│  1. Create flow (deployed_flow.py)                 │
│  2. Deploy it: prefect deploy ...                  │
│     → Tells server about the flow                  │
│     → Server stores deployment metadata            │
└─────────────────────────────────────────────────────┘
                    │
                    │ deployment info
                    ▼
┌─────────────────────────────────────────────────────┐
│  PREFECT SERVER (Docker at :4200)                  │
│                                                     │
│  • Stores deployment definitions                   │
│  • Maintains work pool queues                      │
│  • Accepts flow run requests                       │
│  • Tracks all execution metadata                   │
└─────────────────────────────────────────────────────┘
                    │
                    │ flow runs queued here
                    ▼
┌─────────────────────────────────────────────────────┐
│  WORK POOL: my-local-pool                          │
│                                                     │
│  Queue of pending flow runs waiting to execute     │
└─────────────────────────────────────────────────────┘
                    │
                    │ worker polls for work
                    ▼
┌─────────────────────────────────────────────────────┐
│  WORKER (prefect worker start --pool ...)          │
│                                                     │
│  • Polls work pool every few seconds               │
│  • Picks up flow runs from queue                   │
│  • Executes the flow on its machine                │
│  • Sends progress/logs back to server              │
└─────────────────────────────────────────────────────┘
```

### Key Differences from Before:

| Aspect | Direct Execution | With Deployment |
|--------|-----------------|-----------------|
| **How to run** | `python flow.py` | Server API or schedule |
| **Who triggers** | You manually | Server (on schedule or API call) |
| **Where code runs** | Your terminal | Worker machine |
| **Code location** | Must be local | Must be accessible to worker |
| **Scheduling** | Manual only | Can schedule (cron, interval, etc.) |
| **Production ready** | No | Yes |

### Use Cases:

- **Direct execution** (what we did first): Development, testing, one-off runs
- **Deployments + Workers**: Production, scheduled jobs, distributed execution, team workflows

## 🎓 Try it Yourself!

Here's the complete workflow to see deployments in action:

### 1️⃣ Create the deployment (in a terminal):
```bash
cd /Users/jichong/projects/playground/BOS/prefect-poc
export PREFECT_API_URL=http://localhost:4200/api
uv run prefect deploy deployed_flow.py:data_pipeline --name my-data-pipeline --pool my-local-pool
```

### 2️⃣ Start a worker (in another terminal):
```bash
cd /Users/jichong/projects/playground/BOS/prefect-poc
export PREFECT_API_URL=http://localhost:4200/api
uv run prefect worker start --pool my-local-pool
```

### 3️⃣ Trigger a flow run (in a third terminal or from the UI):
```bash
# Via CLI:
uv run prefect deployment run data-pipeline/my-data-pipeline

# Or open http://localhost:4200/deployments and click "Run"
```

### 4️⃣ Watch it execute!
- The worker terminal will show the flow executing
- The UI at http://localhost:4200 will show real-time progress
- All logs and results are tracked by the server

---

### What you've learned:

✅ **Direct execution** (`python flow.py`):
- Code runs on your machine
- You trigger it manually
- Good for development/testing

✅ **Deployments + Workers**:
- Server orchestrates the execution
- Workers pull jobs from work pools
- Can schedule flows, handle retries, distribute work
- Production-ready approach

This is how Prefect powers production data pipelines at scale! 🚀

# Part 3: Blocks and Remote Storage

## What are Blocks?

**Blocks** are Prefect's way of storing reusable configuration and credentials. Think of them as secure configuration objects that can be:
- Created once and reused across many flows
- Stored on the Prefect server
- Referenced by name in deployments
- Used to connect to external systems (databases, cloud storage, git repos, etc.)

### Common Block Types:
- **Git Repository Blocks**: Connect to GitHub, GitLab, Bitbucket
- **Storage Blocks**: S3, GCS, Azure Blob Storage
- **Secret Blocks**: Store passwords, API keys
- **Infrastructure Blocks**: Docker, Kubernetes, cloud compute

### Why Blocks Matter for Private Bitbucket:
When your code is in a private Git repository:
1. **Worker needs access** to pull the code before running it
2. **Credentials must be secure** (not hardcoded in code)
3. **Blocks solve both problems** by storing Git credentials on the server

## Step 1: Create a Bitbucket Repository Block

Let's create a block that stores credentials for your private Bitbucket server.

In [ ]:
# Create a Bitbucket repository block programmatically

from prefect.blocks.system import Secret
from prefect_bitbucket.credentials import BitBucketCredentials
from prefect_bitbucket.repository import BitBucketRepository

async def create_bitbucket_block():
    """
    Create a Bitbucket repository block for private repo access.
    This block will be stored on the Prefect server.
    """
    
    # Step 1: Create a secret block for the password/token
    # (In production, you'd set this more securely)
    try:
        bitbucket_token = Secret(value="your-app-password-or-token")
        await bitbucket_token.save(name="bitbucket-token", overwrite=True)
        print("✅ Created secret block: bitbucket-token")
    except Exception as e:
        print(f"Note: {e}")
    
    # Step 2: Create credentials block
    try:
        credentials = BitBucketCredentials(
            username="your-username",
            token=bitbucket_token  # Reference to the secret block
        )
        await credentials.save(name="my-bitbucket-creds", overwrite=True)
        print("✅ Created credentials block: my-bitbucket-creds")
    except Exception as e:
        print(f"⚠️  BitBucket block type not installed: {e}")
        print("   You would need to: pip install prefect-bitbucket")
    
    # Step 3: Create repository block
    try:
        repo_block = BitBucketRepository(
            repository="https://bitbucket.mycompany.com/scm/project/repo.git",
            credentials=credentials,
            reference="main"  # branch name
        )
        await repo_block.save(name="my-private-repo", overwrite=True)
        print("✅ Created repository block: my-private-repo")
    except Exception as e:
        print(f"⚠️  Could not create repo block: {e}")
    
    print("\n📝 Block created and stored on Prefect server!")
    print("   Workers can now use this block to pull code from private Bitbucket")

# Note: This will fail without prefect-bitbucket installed
# Showing the pattern for demonstration
print("Example: Creating Bitbucket Block")
print("=" * 60)
print("\n⚠️  Note: This requires 'prefect-bitbucket' to be installed:")
print("   pip install prefect-bitbucket")
print("\nFor demonstration purposes, let's see what blocks we currently have:")

In [18]:
# List all available blocks on the server
async def list_blocks():
    """Query all blocks stored on the Prefect server"""
    async with PrefectClient(api="http://localhost:4200/api") as client:
        # Get block types
        block_types = await client.read_block_types()
        
        print(f"📦 Available Block Types on Server:")
        print(f"   Found {len(block_types)} block types\n")
        
        for bt in block_types[:10]:  # Show first 10
            print(f"   • {bt.name}")
            if bt.description:
                print(f"     {bt.description[:80]}...")
        
        return block_types

blocks = await list_blocks()
print(f"\n💡 These blocks can be used in deployments to store credentials and config")

📦 Available Block Types on Server:
   Found 37 block types

   • AWS Credentials
     Block used to manage authentication with AWS. AWS authentication is
handled via ...
   • Azure
     Store data as a file on Azure Datalake and Azure Blob Storage....
   • Azure Blob Storage Credentials
     Block used to manage Blob Storage authentication with Azure.
Azure authenticatio...
   • Azure Container Instance Credentials
     Block used to manage Azure Container Instances authentication. Stores Azure Serv...
   • Azure Container Instance Job
     Run tasks using Azure Container Instances. Note this block is experimental. The ...
   • Azure Cosmos DB Credentials
     Block used to manage Cosmos DB authentication with Azure.
Azure authentication i...
   • AzureML Credentials
     Block used to manage authentication with AzureML. Azure authentication is
handle...
   • BigQuery Warehouse
     A block for querying a database with BigQuery.

Upon instantiating, a connection...
   • Custom Webhook


## Step 2: How Deployments Work with Private Bitbucket

Here's the complete architecture when using a private Bitbucket server:

### The Flow:

```
┌─────────────────────────────────────────────────────────┐
│  BITBUCKET SERVER (Private)                             │
│  https://bitbucket.mycompany.com                        │
│                                                         │
│  📁 Your Repository:                                    │
│    • flow.py (your flow code)                          │
│    • requirements.txt                                   │
│    • prefect.yaml (deployment config)                  │
└─────────────────────────────────────────────────────────┘
                    ▲
                    │ 1. Git clone with credentials
                    │
┌─────────────────────────────────────────────────────────┐
│  PREFECT SERVER (Your deployment)                       │
│  Could be: Docker, cloud, or on-premises                │
│                                                         │
│  Stores:                                                │
│    • 🔐 Bitbucket credentials (Block)                  │
│    • 📋 Deployment definitions                         │
│    • 🎯 Work pool configurations                       │
│    • 📊 Flow run metadata                              │
└─────────────────────────────────────────────────────────┘
                    │
                    │ 2. Worker polls for work
                    ▼
┌─────────────────────────────────────────────────────────┐
│  WORKER (Running on your infrastructure)                │
│                                                         │
│  When a flow run is scheduled:                          │
│    1. Worker gets job from work pool                   │
│    2. Worker loads Bitbucket credentials from block     │
│    3. Worker clones code from private repo              │
│    4. Worker creates Python environment                 │
│    5. Worker installs dependencies                      │
│    6. Worker executes the flow                          │
│    7. Worker sends results back to server               │
└─────────────────────────────────────────────────────────┘
```

### Key Points:

1. **Prefect server doesn't execute code** - It only stores metadata
2. **Worker pulls code** - Worker machine needs network access to Bitbucket
3. **Credentials stored securely** - Blocks keep secrets on the server
4. **Code is ephemeral** - Worker clones fresh code for each run

## Step 3: Creating a Deployment with Bitbucket Storage

Here's how to deploy a flow that lives in a private Bitbucket repository:

In [ ]:
# Example deployment configuration with Bitbucket storage

deployment_example = """
# prefect.yaml - Deployment configuration file in your Bitbucket repo

name: production-deployment
prefect-version: 2.13.7

# Define where the code comes from
pull:
  - prefect.deployments.steps.git_clone:
      repository: https://bitbucket.mycompany.com/scm/project/myflows.git
      branch: main
      credentials: "{{ prefect.blocks.bitbucket-credentials.my-bitbucket-creds }}"

# Define deployments
deployments:
  - name: prod-data-pipeline
    entrypoint: flows/data_pipeline.py:data_pipeline
    work_pool:
      name: prod-worker-pool
    schedule:
      cron: "0 2 * * *"  # Run at 2 AM daily
    parameters:
      env: production
"""

print("📄 Example prefect.yaml with Bitbucket Storage")
print("=" * 70)
print(deployment_example)

print("\n" + "=" * 70)
print("How to deploy from your Bitbucket repository:")
print("=" * 70)

deployment_steps = """
1️⃣ Push your code to Bitbucket:
   • flows/data_pipeline.py (your flow code)
   • prefect.yaml (deployment config - shown above)
   • requirements.txt (Python dependencies)

2️⃣ Create the Bitbucket credentials block (on Prefect server):
   uv run prefect block register -m prefect_bitbucket
   # Then create the block via UI or Python as shown earlier

3️⃣ Deploy from your local machine (one time):
   cd /path/to/your/repo
   export PREFECT_API_URL=http://your-prefect-server:4200/api
   uv run prefect deploy --all

   This tells the server about your deployment and where to find the code.

4️⃣ Start workers (on your infrastructure):
   # These can run anywhere with access to Bitbucket
   export PREFECT_API_URL=http://your-prefect-server:4200/api
   uv run prefect worker start --pool prod-worker-pool

5️⃣ When a flow run is triggered:
   • Worker gets the job from the server
   • Worker uses the Bitbucket block to clone the repo
   • Worker runs the flow from the cloned code
   • All logs go back to the server
"""

print(deployment_steps)

## Step 4: Answering Your Questions

### Q: "If I deploy my own Prefect server, how will it connect to the private Bitbucket server?"

**Answer: The Prefect server DOESN'T connect to Bitbucket. The WORKER does!**

Here's the breakdown:

#### What the Prefect Server Does:
- ✅ Stores the Bitbucket credentials (in a Block)
- ✅ Stores deployment metadata (which repo, which branch)
- ✅ Manages the work queue
- ❌ **Does NOT** clone code
- ❌ **Does NOT** execute flows
- ❌ **Does NOT** need network access to Bitbucket

#### What the Worker Does:
- ✅ Runs on your infrastructure (can be same server, different server, laptop, cloud VM)
- ✅ **NEEDS** network access to your private Bitbucket server
- ✅ Pulls credentials from the Prefect server (via API)
- ✅ Uses credentials to clone code from Bitbucket
- ✅ Executes the flow
- ✅ Sends results back to Prefect server

### Network Requirements:

```
┌──────────────────────────────────────────────────┐
│  Your Network / VPN                              │
│                                                  │
│  ┌────────────────┐     ┌──────────────────┐   │
│  │ Prefect Server │     │ Bitbucket Server │   │
│  │ :4200          │     │ (Private)        │   │
│  └────────────────┘     └──────────────────┘   │
│         ▲                        ▲              │
│         │                        │              │
│         │  API calls             │ Git clone    │
│         │  (get credentials,     │ (needs auth) │
│         │   report status)       │              │
│         │                        │              │
│  ┌──────┴────────────────────────┴──────┐      │
│  │           WORKER                      │      │
│  │  • Needs access to BOTH servers       │      │
│  │  • Can be same or different machine   │      │
│  └───────────────────────────────────────┘      │
└──────────────────────────────────────────────────┘
```

### Security Best Practices:

1. **Worker Placement**: Deploy workers in the same network zone as Bitbucket
2. **Credentials**: Use SSH keys or app passwords, not your personal password
3. **Least Privilege**: Give the Bitbucket credentials read-only access
4. **Network Policies**: Workers need HTTPS/SSH to Bitbucket, HTTPS to Prefect server
5. **Secrets**: Never commit credentials to git - use Blocks!

## Real-World Example: Production Setup

Let's walk through a complete production setup:

### Scenario:
- Company has private Bitbucket server at `bitbucket.company.com`
- Data engineering team writes flows in `data-team/etl-flows` repo
- Prefect server deployed on `prefect.company.com`
- Workers run on Kubernetes in the same VPC

### Setup Steps:

#### 1. Install Prefect Bitbucket Integration

In [19]:
# Complete production example walkthrough

production_guide = """
================================================================================
PRODUCTION SETUP: Prefect + Private Bitbucket
================================================================================

STEP 1: Install Prefect Bitbucket Integration
----------------------------------------------
On your development machine and in worker Docker images:

    pip install prefect-bitbucket

This provides:
  • BitBucketCredentials block
  • BitBucketRepository block
  • Git clone steps for deployments


STEP 2: Create Bitbucket App Password
--------------------------------------
On bitbucket.company.com:
  1. Go to Personal Settings → App Passwords
  2. Create new app password with 'Repository Read' permission
  3. Save the password securely (you'll need it once)


STEP 3: Register Block Types & Create Blocks
---------------------------------------------
# Register the block types with your Prefect server
export PREFECT_API_URL=https://prefect.company.com/api
prefect block register -m prefect_bitbucket

# Create credentials block (via Python or UI)
from prefect_bitbucket.credentials import BitBucketCredentials
from prefect.blocks.system import Secret

async def setup_blocks():
    # Create secret for the app password
    token = Secret(value="your-app-password-here")
    await token.save("bitbucket-token")
    
    # Create credentials
    creds = BitBucketCredentials(
        username="serviceaccount@company.com",
        token=token
    )
    await creds.save("company-bitbucket-creds")

# Or use the Prefect UI:
# Navigate to Blocks → + → BitBucketCredentials


STEP 4: Structure Your Bitbucket Repository
--------------------------------------------
data-team/etl-flows/
├── flows/
│   ├── daily_etl.py
│   ├── weekly_report.py
│   └── __init__.py
├── requirements.txt
└── prefect.yaml

# requirements.txt
prefect==2.13.7
pandas
sqlalchemy
psycopg2-binary

# prefect.yaml
name: etl-flows
prefect-version: 2.13.7

pull:
  - prefect.deployments.steps.git_clone:
      repository: https://bitbucket.company.com/scm/data-team/etl-flows.git
      branch: "{{ branch }}"
      credentials: "{{ prefect.blocks.bitbucket-credentials.company-bitbucket-creds }}"

deployments:
  - name: daily-etl-prod
    entrypoint: flows/daily_etl.py:main_flow
    work_pool:
      name: k8s-prod-pool
    schedule:
      cron: "0 1 * * *"
    parameters:
      environment: production


STEP 5: Deploy from Your Machine (One Time)
--------------------------------------------
cd /path/to/etl-flows
export PREFECT_API_URL=https://prefect.company.com/api
prefect deploy --all

This registers the deployment with the server. The server now knows:
  • Where to find the code (Bitbucket URL)
  • Which credentials to use (the block name)
  • Which branch to use (main)
  • Schedule and parameters


STEP 6: Run Workers (Continuously)
-----------------------------------
Workers run on your infrastructure (K8s, VMs, etc)

# Kubernetes deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: prefect-worker
spec:
  replicas: 3
  template:
    spec:
      containers:
      - name: worker
        image: prefecthq/prefect:2.13.7-python3.10
        command: ["prefect", "worker", "start"]
        args: ["--pool", "k8s-prod-pool"]
        env:
        - name: PREFECT_API_URL
          value: "https://prefect.company.com/api"

Workers will:
  1. Poll the work pool for jobs
  2. When a job arrives, load credentials from the server
  3. Clone code from Bitbucket using those credentials
  4. Execute the flow
  5. Report results back


STEP 7: Trigger & Monitor
--------------------------
Flows run automatically on schedule, or trigger manually:

  prefect deployment run daily-etl-prod/daily-etl-prod

Monitor at: https://prefect.company.com/deployments


SECURITY NOTES:
--------------
✅ Credentials never in code - stored in Blocks on server
✅ Workers pull fresh code each run - no stale code
✅ Use service account with read-only Bitbucket access
✅ Workers need network access to both Prefect server AND Bitbucket
✅ Prefect server only needs to be accessible to workers (not Bitbucket)
"""

print(production_guide)


PRODUCTION SETUP: Prefect + Private Bitbucket

STEP 1: Install Prefect Bitbucket Integration
----------------------------------------------
On your development machine and in worker Docker images:

    pip install prefect-bitbucket

This provides:
  • BitBucketCredentials block
  • BitBucketRepository block
  • Git clone steps for deployments


STEP 2: Create Bitbucket App Password
--------------------------------------
On bitbucket.company.com:
  1. Go to Personal Settings → App Passwords
  2. Create new app password with 'Repository Read' permission
  3. Save the password securely (you'll need it once)


STEP 3: Register Block Types & Create Blocks
---------------------------------------------
# Register the block types with your Prefect server
export PREFECT_API_URL=https://prefect.company.com/api
prefect block register -m prefect_bitbucket

# Create credentials block (via Python or UI)
from prefect_bitbucket.credentials import BitBucketCredentials
from prefect.blocks.system import

## Summary: Key Takeaways

### 🔑 Blocks Explained:
- **What**: Reusable configuration objects stored on Prefect server
- **Why**: Securely store credentials, avoid hardcoding secrets
- **How**: Create once, reference by name in deployments
- **Types**: Git repos, databases, cloud storage, secrets, infrastructure

### 🔒 Private Bitbucket + Prefect Architecture:

| Component | Role | Needs Access To |
|-----------|------|-----------------|
| **Bitbucket Server** | Stores your flow code | Nothing (it's the source) |
| **Prefect Server** | Stores metadata, credentials, schedules | Nothing (workers connect to it) |
| **Worker** | Executes flows | **Both** Bitbucket AND Prefect Server |

### 📊 The Complete Flow:

1. **Developer** pushes code to private Bitbucket
2. **Developer** creates Bitbucket credentials Block on Prefect server
3. **Developer** runs `prefect deploy` to register deployment
4. **Prefect Server** stores deployment metadata (not the code!)
5. **Worker** polls Prefect server for work
6. **Worker** gets credentials from Prefect server (via Block)
7. **Worker** clones code from Bitbucket using those credentials
8. **Worker** executes the flow
9. **Worker** sends results back to Prefect server
10. **You** view results in Prefect UI

### ✅ What You've Learned:

1. **Blocks** are Prefect's way to store reusable config and credentials
2. **Prefect server** never executes code or clones repos
3. **Workers** do all the heavy lifting (clone + execute)
4. **Network access**: Workers need access to both Prefect server and Bitbucket
5. **Security**: Credentials in Blocks, never in code
6. **Deployment**: Code stays in git, workers pull it fresh each time

### 🚀 Next Steps:

1. Install `prefect-bitbucket`: `pip install prefect-bitbucket`
2. Create a service account in Bitbucket with read-only access
3. Create Bitbucket credentials Block on your Prefect server
4. Add `prefect.yaml` to your repository
5. Deploy: `prefect deploy --all`
6. Start workers in your infrastructure
7. Watch flows run from your private Bitbucket repo!

---

**Remember**: The beauty of this architecture is that your Prefect server can be anywhere (cloud, on-prem, DMZ), and your workers just need network access to both the server and Bitbucket. The server itself never needs to reach Bitbucket! 🎯